In [ ]:
# Install required libraries.
# pip install moviepy opencv-python ffmpeg transformers pytorch torchvision speechrecognition librosa huggingface_hub
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

### Extract video components

In [4]:
from moviepy import VideoFileClip

def extract_audio_and_frames(video_path, frame_rate=1):
    clip = VideoFileClip(video_path)
    # Extract audio
    audio_path = "audio.wav"
    clip.audio.write_audiofile(audio_path)
    # Extract frames
    frames = []
    for t in range(0, int(clip.duration), frame_rate):
        frame = clip.get_frame(t)
        frames.append(frame)
    return audio_path, frames

### Analyze audio

In [8]:
import whisper
from transformers import pipeline

# Transcribe audio
def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result['text']

# Explicit language detection
def detect_explicit_language(text):
    classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")
    results = classifier(text)
    return [res for res in results if res['label'] == 'EXPLICIT']


d:\src_git\curiosity\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Analyze Video Frames

In [9]:
from ultralytics import YOLO

# Detect explicit visuals in frames
def detect_explicit_frames(frames):
    model = YOLO('yolov8n.pt')  # Replace with a fine-tuned model
    explicit_frames = []
    for idx, frame in enumerate(frames):
        results = model.predict(frame)
        if "explicit" in [res['name'] for res in results.xyxy[0]]:
            explicit_frames.append(idx)
    return explicit_frames
    

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\kumar\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Combine results

In [10]:
def analyze_video(video_path):
    audio_path, frames = extract_audio_and_frames(video_path)
    transcription = transcribe_audio(audio_path)
    explicit_text = detect_explicit_language(transcription)
    explicit_frames = detect_explicit_frames(frames)
    return {"explicit_text": explicit_text, "explicit_frames": explicit_frames}


### Save Results

In [11]:
def save_results(results, output_path="explicit_timings.txt"):
    with open(output_path, "w") as f:
        for key, value in results.items():
            f.write(f"{key}: {value}\n")


### Testing the script

In [ ]:
if __name__ == "__main__":
    video_path = "test_video.mp4"  # Replace with your video file
    results = analyze_video(video_path)
    save_results(results)
    print("Analysis complete. Results saved to explicit_timings.txt.")
